### Install Library

In [ ]:
pip install -U accelerate==0.29.3 peft==0.10.0 bitsandbytes==0.43.1 transformers==4.40.1 trl==0.8.6  datasets==2.19.0

In [ ]:
import os
import torch
from datasets import load_dataset

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer

In [ ]:
import huggingface_hub
huggingface_hub.login()

# 1. set model and dataset

★ set point:  set your own model path in huggingface★

In [ ]:
# set base model path
base_model = "beomi/Llama-3-Open-Ko-8B"
#base_model = "meta-llama/Meta-Llama-3-8B-Instruct"
# set new model path
new_model = "Llama3-rs2-Ko-8b"
#new_model = "Llama3-rs2-meta-8b"

In [ ]:
dataset_namehk = "rssaem/testdb2"
datasethk = load_dataset(dataset_namehk, split="train")

In [ ]:
def create_text_column(example):
    # 'text' 컬럼 생성
    text = f"### Instruction:\n{example['instruction']}\n\n### Response:\n{example['output']}"
    example["text"] = text
    return example

# 'text' 컬럼 생성
datasetrs = datasethk.map(create_text_column)

In [ ]:
datasetrs

Dataset({
    features: ['instruction', 'output', 'input', 'text'],
    num_rows: 41
})

In [ ]:
datasetrs[7]

{'instruction': 'BTS 음악의 컨셉은 무엇입니까?',
 'output': '전반적으로 10대와 20대 청춘들의 생각과 고민, 삶과 사랑, 꿈과 역경을 주요 주제로 하는 노래들을 통해 자신들만의 세계관을 구축하고 있으며, 연계되는 이야기를 다양한 뮤직비디오들을 통해 유기적으로 풀어 나가는 모습을 보여준다.',
 'input': '',
 'text': '### Instruction:\nBTS 음악의 컨셉은 무엇입니까?\n\n### Response:\n전반적으로 10대와 20대 청춘들의 생각과 고민, 삶과 사랑, 꿈과 역경을 주요 주제로 하는 노래들을 통해 자신들만의 세계관을 구축하고 있으며, 연계되는 이야기를 다양한 뮤직비디오들을 통해 유기적으로 풀어 나가는 모습을 보여준다.'}

# 2. Config efficient fine-tuning with low-rank adaptation.

In [ ]:
print(torch.cuda.get_device_capability()[0])

8


In [ ]:
# 현재 사용 중인 GPU의 주요 아키텍처 버전을 반환 8버전 이상 시 bfloat16 활용
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install -qqq flash-attn
    attn_implementation = "flash_attention_2"
    torch_dtype = torch.bfloat16
else:
    attn_implementation = "eager"
    torch_dtype = torch.float16

# BitsAndBytesConfig 객체활용 양자화 설정
# 모델을 4비트 양자화하여 로드
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=False,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


# 3. Load Pre-trained Language Model

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map={"": 0}
    # device_map="auto"
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# 4. Load Pre-trained Language Model Tokenizer

In [ ]:
# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
# Must add EOS_TOKEN at response last line
tokenizer.pad_token = tokenizer.eos_token
# ★수정 포인트!!! 기존 # tokenizer.padding_side = "right"
EOS_TOKEN = tokenizer.eos_token
def prompt_eos(sample):
    sample['text'] = sample['text']+EOS_TOKEN
    return sample
datasetrs = datasetrs.map(prompt_eos)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/41 [00:00<?, ? examples/s]

In [ ]:
datasethk[1]

# 5. Config training parameter for LoRA (Parameter-Efficient Fine-Tuning (PEFT)

https://huggingface.co/docs/peft/conceptual_guides/lora

In [ ]:
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=100,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


# 6. Train Model

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=datasethk,
    peft_config=peft_params,
    dataset_text_field="text",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)
trainer.train()

# 7. Verify

In [ ]:
 def generate_response(prompt, model):
    encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
    model_inputs = encoded_input.to('cuda')

    generated_ids = model.generate(**model_inputs, max_new_tokens=512, do_sample=True, pad_token_id=tokenizer.eos_token_id)
    # generated_ids = model.generate(**model_inputs, max_new_tokens=200, do_sample=True, pad_token_id=128009)

    decoded_output = tokenizer.batch_decode(generated_ids)

    return decoded_output[0].replace(prompt, "")
key="BTS는 누구인가요?"
prompt=f"""you are a assistant please answer in korean lanauage

### Instruction:
{key}

### Response:"""
generate_response(prompt, model)

'<|begin_of_text|> \n\nBTS는 2013년 한림대 출신의 7명의 학생들이 창단한 팀이다. 2013년 UN 정기총회 등 발달금 등 행사에서 발휘한 덕에 세계적으로 주목받아 전 세계인들의 응원받는 글로벌 팀이 되었다. 2020년 9월 12일 UN 제75회 총회 연설자로 선정되어 세계인들에게 메시지를 전달하는 기회를 갖게 되었다. 이후에도 연이은 유기기금 받아 글로벌 팀으로 성장하였고, 2022년 6월 10일 미군 후원 기금으로 또다시 세계인들의 응원받는 글로벌 팀이 되었다. 2022년 7월 16일 빌보드 뮤직 어워즈에 수석 부대표로 선정되어 또다시 세계인들의 응원받는 글로벌 팀이 되었다. 2022년 9월 18일 영국 왕립예술협회(Royal Society of Arts, RSA) 회원으로 가입하여 또다시 세계인들의 응원받는 글로벌 팀이 되었다. 2022년 10월 7일 유니세프 청년대표로 선정되어 또다시 세계인들의 응원받는 글로벌 팀이 되었다. 2022년 10월 14일 미국 국립블루스기금 대표로 선정되어 또다시 세계인들의 응원받는 글로벌 팀이 되었다. 2013년 UN 정기총회 등 발달금 등 행사에서 발휘한 덕에 세계적으로 주목받아 전 세계인들의 응원받는 글로벌 팀이 되었다. 이후에도 연이은 유기기금 받아 글로벌 팀으로 성장하였다. 2022년 6월 10일 미군 후원 기금으로 또다시 세계인들의 응원받는 글로벌 팀이 되었다. 2022년 7월 16일 빌보드 뮤직 어워즈에 수석 부대표로 선정되어 또다시 세계인들의 응원받는 글로벌 팀이 되었다. 2022년 9월 18일 영국 왕립예술협회(Royal Society of Arts, RSA) 회원으로 가입하여 또다시 세계인들의 응원'

In [ ]:
#텍스트 생성을 위한 pipeline 생성 (모델 이름은 실제 모델로 변경 필요)
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)  # GPU 사용

key = "BTS 음악의 컨셉은 무엇입니까?"
prompt = f"""you are a assistant please answer in korean lanauage

### Instruction:
{key}

### Response:"""

response = generator(prompt, max_length=200, do_sample=True, pad_token_id=tokenizer.eos_token_id)

# 결과 출력
print(response[0]['generated_text'].replace(prompt, ""))

 자신만의 세계를 살아가는 일곱 젊이들이 겪는 일을 보고 느낀 생각들을 직설적으로 연주와 노래로 풀어내는 것이다. 우승이 좌절된 이후 처음 출전한 국제무대에서 또 한 번 우승을 따낸 선수는 바로 정경희(56)다. 1988년 서울올림픽 여자단식 정상에 오른 뒤 90년 자국에서 열린 대회에서도 관록을 보이며 금메달을 추가했던 정경희는 1992년 바르셀로나올림픽에서 동메달을 목에 걸었다.
 정경희의 후배 이상근(53)은 1996년 애틀랜타올림픽 단식에서 금메달을 차지


In [ ]:
savePath = "finetuning/llama3_rs_meta_1013_100"
trainer.save_model(savePath)